# Wave Energy
Notebook environment to migrate netcdf files to CF compliant zarr

In [1]:
# Optional; code formatter, installed as jupyter lab extension
#%load_ext lab_black
# Optional; code formatter, installed as jupyter notebook extension
%load_ext nb_black

<IPython.core.display.Javascript object>

### Configure OS independent paths

In [2]:
# Import standard packages
import os
import pathlib
import sys
import numpy as np
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import netCDF4 as nc
import numpy.ma as ma

# Make root directories importable by appending root to path
cwd = pathlib.Path().resolve()
sys.path.append(os.path.dirname(cwd))
print(os.path.dirname(cwd))
# Get root paths
home = pathlib.Path().home()
root = home.root

# Import custom functionality
from etl import p_drive
from etl.CF_compliancy_checker import check_compliancy, save_compliancy

# Define (local and) remote drives
coclico_data_dir = p_drive.joinpath("11205479-coclico", "data")

# Workaround to the Windows OS (10) udunits error after installation of cfchecker: https://github.com/SciTools/iris/issues/404
os.environ["UDUNITS2_XML_PATH"] = str(
    home.joinpath(  # change to the udunits2.xml file dir in your Python installation
        r"Anaconda3\pkgs\udunits2-2.2.28-h892ecd3_0\Library\share\udunits\udunits2.xml"
    )
)

C:\Users\heijd_de\AppData\Local\Temp\ipykernel_2792\1556717457.py:6: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


\\storage005.directory.intra\1000545-054-globalbeaches\15_GlobalCoastalAtlas\coclicodata


<IPython.core.display.Javascript object>

In [3]:
# Project paths & files (manual input)
ds_dir = coclico_data_dir.joinpath("02_wave_energy_jrc")
ds_rcp85_path = ds_dir.joinpath("CoastAlRisk_Global_WEF_RCP85.nc")
ds_out_file = "CoastAlRisk_Global_WEF_RCP85"
CF_dir = coclico_data_dir.joinpath(r"CF")  # directory to save output CF check files

<IPython.core.display.Javascript object>

### Check CF compliancy original NetCDF files

In [4]:
# open datasets
ds_85rcp = xr.open_dataset(ds_rcp85_path)

# check original dataset
ds_85rcp

<xarray.Dataset>
Dimensions:    (npoints: 4828, nrp: 8, nsdec: 11)
Dimensions without coordinates: npoints, nrp, nsdec
Data variables:
    longitude  (npoints) float64 ...
    latitude   (npoints) float64 ...
    wef        (npoints, nrp, nsdec) float64 ...
    rp         (nrp) float32 ...
    decades    (nsdec) float32 ...
Attributes:
    title:            Global Wave Energy Flux
    Institution:      Joint European Research Center, Directorate for Space, ...
    Project Name:     Prototype of a first Global Integrated Coastal Impact-b...
    Project Acronym:  CoastAlRisk
    reference:        Mentaschi L., Vousdoukas M., Voukouvalas E., Dosio A., ...
    email:            lorenzo.mentaschi@ec.europa.eu
    version:          1.0
    terms_for_use:    European Union, 1995-2015.\nReuse is authorised, provid...
    disclaimer:       Unless the following would not be permitted or valid un...

<IPython.core.display.Javascript object>

In [6]:
ds_85rcp.nsdec

<xarray.DataArray 'nsdec' (nsdec: 11)>
array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10], dtype=int64)
Dimensions without coordinates: nsdec

<IPython.core.display.Javascript object>

In [7]:
%%capture cap --no-stderr
# check original CF compliancy

check_compliancy(testfile=ds_rcp85_path, working_dir=CF_dir)

<IPython.core.display.Javascript object>

In [8]:
# save original CF compliancy
save_compliancy(cap, testfile=ds_rcp85_path, working_dir=CF_dir)

{'P:\\11205479-coclico\\data\\02_wave_energy_jrc\\CoastAlRisk_Global_WEF_RCP85.nc': {'warnings': '3', 'errors': '2'}}


<IPython.core.display.Javascript object>

### Make CF compliant alterations to the NetCDF files (dataset dependent)

In [9]:
# NetCDF attribute alterations

# rename global attribute names
ds_85rcp.attrs["Project_Name"] = ds_85rcp.attrs.pop("Project Name")
ds_85rcp.attrs["Project_Acronym"] = ds_85rcp.attrs.pop("Project Acronym")

# add global attributes
ds_85rcp.attrs["Conventions"] = "CF-1.8"
ds_85rcp.attrs["Starting_date"] = "01-Dec-2009"
ds_85rcp.attrs["End_date"] = "30-Nov-2099 21:00:00"

# remove certain variable attributes
del ds_85rcp["rp"].attrs["Starting date"]
del ds_85rcp["rp"].attrs["End date"]

# add or change certain variable / coordinate attributes
dataset_attributes = {
    "decades": {"long_name": "decade window", "Format": "YYYY", "units": "yr"}
}  # specify custom (CF convention) attributes

# add / overwrite attributes
for k, v in dataset_attributes.items():
    try:
        ds_85rcp[k].attrs = dataset_attributes[k]
    except:
        continue

<IPython.core.display.Javascript object>

In [10]:
# NetCDF variable and dimension alterations

# rename or swap dimension names, the latter in case the name already exists as coordinate
ds_85rcp = ds_85rcp.rename_dims({"npoints": "stations", "nsdec": "time"})
ds_85rcp = ds_85rcp.swap_dims({"nrp": "rp"})

# rename variables, if necessary
ds_85rcp = ds_85rcp.rename_vars(
    {"longitude": "lon", "latitude": "lat", "decades": "time"}
)

# set some data variables to coordinates to avoid duplication of dimensions in later stage
ds_85rcp = ds_85rcp.set_coords(["lon", "lat", "rp", "time"])

<IPython.core.display.Javascript object>

In [11]:
# re-order shape of the data variables
ds_85rcp = ds_85rcp.transpose("stations", "rp", "time")

# add epsg
ds_85rcp.attrs["crs"] = 4326

<IPython.core.display.Javascript object>

In [12]:
# check the xarray dataset, best practice is to have as many as possible bold dimensions (dimension == coordinate name).
# in this way, the Front-End can access the variable directly without having to index the variable first

ds_85rcp['rp']

<xarray.DataArray 'rp' (rp: 8)>
array([   5.,   10.,   20.,   50.,  100.,  200.,  500., 1000.], dtype=float32)
Coordinates:
  * rp       (rp) float32 5.0 10.0 20.0 50.0 100.0 200.0 500.0 1e+03
Attributes:
    long_name:  return period
    units:      years

<IPython.core.display.Javascript object>

In [13]:
ds_85rcp

<xarray.Dataset>
Dimensions:  (stations: 4828, rp: 8, time: 11)
Coordinates:
    lon      (stations) float64 -180.0 -180.0 -180.0 ... 181.3 181.5 181.6
    lat      (stations) float64 71.87 69.81 71.87 51.38 ... -18.52 -17.54 -16.57
  * rp       (rp) float32 5.0 10.0 20.0 50.0 100.0 200.0 500.0 1e+03
    time     (time) float32 1.995e+03 2.01e+03 2.02e+03 ... 2.09e+03 2.1e+03
Dimensions without coordinates: stations
Data variables:
    wef      (stations, rp, time) float64 ...
Attributes: (12/13)
    title:            Global Wave Energy Flux
    Institution:      Joint European Research Center, Directorate for Space, ...
    reference:        Mentaschi L., Vousdoukas M., Voukouvalas E., Dosio A., ...
    email:            lorenzo.mentaschi@ec.europa.eu
    version:          1.0
    terms_for_use:    European Union, 1995-2015.\nReuse is authorised, provid...
    ...               ...
    Project_Name:     Prototype of a first Global Integrated Coastal Impact-b...
    Project_Acronym:  CoastAlRisk
    Conventions:      CF-1.8
    Starting_date:    01-Dec-2009
    End_date:         30-Nov-2099 21:00:00
    crs:              4326

<IPython.core.display.Javascript object>

In [11]:
# save new .nc files
ds_85rcp.to_netcdf(path=ds_dir.joinpath(ds_out_file + "_CF.nc"))

<IPython.core.display.Javascript object>

### Check CF compliancy altered NetCDF files

In [12]:
%%capture cap --no-stderr
# check altered CF compliancy

check_compliancy(testfile=ds_dir.joinpath(ds_out_file + "_CF.nc"), working_dir=CF_dir)

<IPython.core.display.Javascript object>

In [13]:
# save altered CF compliancy
save_compliancy(
    cap, testfile=ds_dir.joinpath(ds_out_file + "_CF.nc"), working_dir=CF_dir,
)

{'P:\\11205479-coclico\\data\\02_wave_energy_jrc\\CoastAlRisk_Global_WEF_RCP85_CF.nc': {'warnings': '0', 'errors': '0'}}


<IPython.core.display.Javascript object>

### write data to Zarr files

In [14]:
# export to zarr in write mode (to overwrite if exists)
ds_85rcp.to_zarr(ds_dir.joinpath("%s.zarr" % ds_out_file), mode="w")

<IPython.core.display.Javascript object>